# HateXplain

https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two


The model is used for classifying a text as Abusive (Hatespeech and Offensive) or Normal. The model is trained using data from Gab and Twitter and Human Rationales were included as part of the training data to boost the performance. The model also has a rationale predictor head that can predict the rationales given an abusive sentence.

The dataset and models are available here: https://github.com/punyajoy/HateXplain


Binny Mathew, Punyajoy Saha, Seid Muhie Yimam, Chris Biemann, Pawan Goyal, and Animesh Mukherjee "[HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection)". Accepted at AAAI 2021.


@article{mathew2020hatexplain,
  title={HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection},
  author={Mathew, Binny and Saha, Punyajoy and Yimam, Seid Muhie and Biemann, Chris and Goyal, Pawan and Mukherjee, Animesh},
  journal={arXiv preprint arXiv:2012.10289},
  year={2020}
}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
### from models.py
from models import *
import os
import shutil
from datetime import date

import pandas as pd
import numpy as np
import torch

from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

seed = 42
batch_size = 1
MODEL_NAME = "HateXplain"

# Loading model 
It is in the cuda device if is avalible.

Classifies sentences into "hate speech", "normal", "offensive"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain").to(device)

# Loading dataset
This is the hatexplain dataset, with sentence tokens, each annotators labels, reasoning, and hate targets. 

In this situaiton we are only using the sentence tokens and labels as a standarized benchmark dataset to comapre across models

This model was trianed on this dataset, so the benchmarking quality in this context is not as assured as we'd like

Data is tokenized and batched here

In [ ]:
dataset = load_dataset("hatexplain", split="validation")

class_names = ["hate speech", "normal", "offensive"]

data_tokens = tokenizer(dataset['post_tokens'], padding=True,is_split_into_words=True, return_tensors="pt")


input_tensor = []
attention_mask_tensor = []
for i in range(0,len(data_tokens.input_ids), batch_size):
    input_tensor += [data_tokens.input_ids[i:i+batch_size]]
    attention_mask_tensor += [data_tokens.attention_mask[i:i+batch_size]]

# Inference

Data is loaded into cuda/device, the model then runs a prediction, then results are unpacked and runs softmax(ed).

We do this becuse all the data and model combined are >8GB in memory, so we need to load and unload data continuously

also the softmax is done in cuda/device/pytorch, its convinient to do here

In [ ]:
predicitons = []
for i in range(len(input_tensor)):
    with torch.no_grad():
        inp = input_tensor[i].to(device)
        attention = attention_mask_tensor[i].to(device)
        prediction_logits = model.forward(input_ids=inp,attention_mask=attention).logits

        for x in prediction_logits: # for each batch per output 
            predicitons.append(torch.argmax(x, dim=0).item())

        inp.detach()
        attention.detach()

# pack predictions and gold/true lables together

I have not 100% sure if the true gold labels match with thier coresponding predicitons. I think they do.

data


In [ ]:
predictions_with_labels = []
for i in range(len(predicitons)):
    predictions_with_labels.append((predicitons[i], int(np.median(dataset[i]['annotators']['label']))))

In [ ]:
print(predictions_with_labels[0])
predction_data = pd.DataFrame(predictions_with_labels, columns=["prediciton", "annotators_labels"])
predction_data.to_csv(path_or_buf=f"outputs\\{MODEL_NAME}_{date.today()}.csv", index=False)
print(predction_data)

In [ ]:
print(f"dataset size: {len(predictions_with_labels)}")
for i in range(len(predictions_with_labels)):
    print(
        f"data: {' '.join(dataset[i]['post_tokens'])} " +
        f"\n\tpredictiton: {predictions_with_labels[i][0]} {class_names[predictions_with_labels[i][0]]} "+
        f"\n\trecorded label: {predictions_with_labels[i][1]}"+
        f"\n\ttrue lable: {dataset[i]['annotators']['label']} {class_names[int(np.median(dataset[i]['annotators']['label']))]}\n")